In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('data/cards.csv')    
prices = pd.read_csv('data/prices.csv')

pd.options.display.max_columns = data.shape[1] #display option

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,9,12,16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data['text'] = data['text'].astype(str)

In [4]:
data.text.map(lambda x: len(x)).max()

769

In [5]:
df = data['text'].apply(lambda x: len(x))

In [6]:
df.describe(percentiles = [.25, .5, .75, .8, .9, .95])

count    51430.000000
mean       127.182306
std         85.638511
min          3.000000
25%         63.000000
50%        113.000000
75%        178.000000
80%        196.000000
90%        246.000000
95%        287.000000
max        769.000000
Name: text, dtype: float64

In [7]:
data.columns

Index(['index', 'id', 'artist', 'asciiName', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorName', 'flavorText',
       'frameEffect', 'frameEffects', 'frameVersion', 'hand', 'hasFoil',
       'hasNoDeckLimit', 'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox',
       'isDateStamped', 'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized',
       'isPaper', 'isPromo', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'layout',
       'leadershipSkills', 'life', 'loyalty', 'manaCost', 'mcmId', 'mcmMetaId',
       'mtgArenaId', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'names',
       'number', 'originalText', 'originalType', 'otherFaceIds', 'power',
       'printings', 'purchaseUrls', 'rarity', 'scryfallId',
       'scryfallIllustrationId', 'scryfallOracleId', 'setCode', 'side',
       'subtypes', 'supertypes', 'tcgplayerProduc

In [8]:
df.loc[df == 769].head()

47238    769
Name: text, dtype: int64

In [9]:
data.iloc[47238]['name']

'Bureaucracy'

In [10]:
data['life'].fillna(data['life'].mean(), inplace=True)

In [11]:
data['life']

0        1.135593
1        1.135593
2        1.135593
3        1.135593
4        1.135593
           ...   
51425    1.135593
51426    1.135593
51427    1.135593
51428    1.135593
51429    1.135593
Name: life, Length: 51430, dtype: float64

In [12]:
# data['power'] = data['power'].astype(float)
data['power'].apply(np.isreal)

0         True
1        False
2         True
3         True
4         True
         ...  
51425     True
51426    False
51427    False
51428     True
51429    False
Name: power, Length: 51430, dtype: bool

In [13]:
data.iloc[2]['name']

'Adarkar Wastes'

In [14]:
# data['power'].fillna(data['power'].mean(), inplace=True)
data['power'] = pd.to_numeric(data['power'], errors='coerce')

In [15]:
data['power'].fillna(data['power'].mean(), inplace=True)

In [16]:
data['power']

0        2.614963
1        2.000000
2        2.614963
3        2.614963
4        2.614963
           ...   
51425    2.614963
51426    2.000000
51427    4.000000
51428    2.614963
51429    3.000000
Name: power, Length: 51430, dtype: float64

In [17]:
data.head()

,index,id,artist,asciiName,borderColor,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,flavorName,flavorText,frameEffect,frameEffects,frameVersion,hand,hasFoil,hasNoDeckLimit,hasNonFoil,isAlternative,isArena,isBuyABox,isDateStamped,isFullArt,isMtgo,isOnlineOnly,isOversized,isPaper,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgoFoilId,mtgoId,multiverseId,name,names,number,originalText,originalType,otherFaceIds,power,printings,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,0,1,Rebecca Guay,NaN,black,G,NaN,G,4.0,NaN,1011.0,NaN,NaN,NaN,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,1.135593,NaN,{2}{G}{G},16413.0,19.0,NaN,27283.0,27282.0,130483.0,Abundance,NaN,249,"If you would draw a card, you may instead choo...",Enchantment,NaN,2.614963,"10E,C17,DDR,USG",{'cardmarket': 'https://mtgjson.com/links/2b93...,rare,46184f97-d5c9-4a98-9fd9-e19057ce9b7e,4be12219-df08-4a40-b32f-69737db62615,eccc9a54-2b56-4a02-9926-258d5b2e25fb,10E,NaN,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,1669af17-d287-5094-b005-4b143441442f,NaN,NaN
1,1,2,Stephen Daniele,NaN,black,U,NaN,U,3.0,NaN,10859.0,NaN,NaN,They brandish their latest theories as warrior...,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,1.135593,NaN,{1}{U}{U},16227.0,26.0,NaN,27275.0,27274.0,132072.0,Academy Researchers,NaN,63,"When Academy Researchers comes into play, you ...",Creature - Human Wizard,NaN,2.000000,"10E,USG",{'cardmarket': 'https://mtgjson.com/links/27c7...,uncommon,b62ca800-ec24-476e-b56f-834eef9622a9,cf5efc65-9aa2-466b-bd2b-448ad69ab474,df2d2c52-475b-4057-9cf8-7886700f0fae,10E,NaN,"Human,Wizard",NaN,15024.0,When Academy Researchers enters the battlefiel...,2,Creature — Human Wizard,Creature,047d5499-a21c-5f5c-9679-1599fcaf9815,NaN,NaN
2,2,3,John Avon,NaN,black,"U,W",NaN,NaN,0.0,NaN,570.0,NaN,NaN,NaN,NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,1.135593,NaN,NaN,16511.0,42.0,NaN,27385.0,27384.0,129458.0,Adarkar Wastes,NaN,347,{T}: Add {1} to your mana pool.\n{T}: Add {W} ...,Land,NaN,2.614963,"10E,5ED,6ED,7ED,9ED,ICE,PTC,WC00",{'cardmarket': 'https://mtgjson.com/links/7866...,rare,f31cf014-7ac9-428b-9ce9-8ba5ebfdd252,1b3845b2-2bac-4cfe-a0c5-6e9f4bd1edb8,d5ad26cc-2bdb-46b7-b8bf-dd099d5fa09b,10E,NaN,NaN,NaN,15025.0,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,NaN,Land,Land,ee19938c-4007-58f1-8904-fae28007b422,NaN,NaN
3,3,4,Roger Raupp,NaN,black,B,NaN,B,3.0,NaN,16135.0,NaN,NaN,"One rarely notices a heartbeat, save when it i...",NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,1.135593,NaN,{2}{B},16289.0,66.0,NaN,27483.0,27482.0,135206.0,Afflict,NaN,125,Target creature gets -1/-1 until end of turn.\...,Instant,NaN,2.614963,"10E,ODY",{'cardmarket': 'https://mtgjson.com/links/02e8...,common,20f18447-2853-469b-b3b3-20d6bcafa6e7,83259a38-bbad-4dd1-bcd5-d1b1d96b0380,471f6ad3-cf01-42dd-a519-ea8e13039fea,10E,NaN,NaN,NaN,15026.0,Target creature gets -1/-1 until end of turn.\...,NaN,Instant,Instant,8774e18f-3752-5c06-af94-5da3960da9ed,NaN,NaN
4,4,5,Christopher Moeller,NaN,black,G,NaN,G,2.0,NaN,9933.0,NaN,NaN,"The power of the wild, concentrated in a singl...",NaN,NaN,2003,NaN,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,normal,NaN,1.135593,NaN,{1}{G},16414.0,76.0,NaN,27361.0,27360.0,130525.0,Aggressive Urge,NaN,250,Target creature gets +1/+1 until end of turn.\...,Instant,NaN,2.614963,"10E,INV,MB1,RIX",{'cardmarket': 'https://mtgjson.com/links/2ec1...,common,bf92ae01-9d1e-4b41-b068-096648daadf6,f4a2d3c4-15b8-4808-a5c5-56aaefc0dfcc,43f5a93b-0f8d-48d2-ab9d-275d44cf88b5,10E,NaN,NaN,NaN,15027.0,Target creature gets +1/+1 until end of turn.\...,NaN,Instant,Instant,4e875bca-0c52-5d60-889d-1d

In [18]:
data['convertedManaCost']

0        4.0
1        3.0
2        0.0
3        3.0
4        2.0
        ... 
51425    4.0
51426    4.0
51427    5.0
51428    2.0
51429    5.0
Name: convertedManaCost, Length: 51430, dtype: float64

In [19]:
data['colors']

0          G
1          U
2        NaN
3          B
4          G
        ... 
51425      W
51426      U
51427      W
51428      R
51429      G
Name: colors, Length: 51430, dtype: object

In [20]:

pd.get_dummies(data['colors'])

,B,"B,G","B,G,R","B,G,R,U","B,G,R,U,W","B,G,R,W","B,G,U","B,G,U,W","B,G,W","B,R","B,R,U","B,R,U,W","B,R,W","B,U","B,U,W","B,W",G,"G,R","G,R,U","G,R,U,W","G,R,W","G,U","G,U,W","G,W",R,"R,G","R,U","R,U,W","R,W",U,"U,B","U,G","U,R","U,W",W,"W,B","W,G","W,R","W,U"
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51425,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
51426,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
51427,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
51428,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
pd.options.display.max_columns = 691
pd.get_dummies(data['manaCost'])

,{0},{1000000},{10},{10}{B},{10}{B}{B},{10}{G},{10}{G}{G},{10}{G}{G}{G}{W}{W},{11},{12},{12}{U}{U},{13},{15},{16},{1},{1}{2/U}{2/R},{1}{B/G},{1}{B/G}{B/G},{1}{B/G}{B/G}{B/G},{1}{B/P},{1}{B/P}{B/P},{1}{B/R},{1}{B/R}{B/R},{1}{B/R}{B/R}{B/R},{1}{B},{1}{B}{B},{1}{B}{B}{B},{1}{B}{B}{G},{1}{B}{G},{1}{B}{G}{U},{1}{B}{R},{1}{B}{R}{G},{1}{B}{R}{R},{1}{C},{1}{G/U},{1}{G/U}{G/U},{1}{G/W},{1}{G/W}{G/W},{1}{G/W}{G/W}{G/W},{1}{G},{1}{G}{G},{1}{G}{G}{G},{1}{G}{G}{U},{1}{G}{G}{W},{1}{G}{G}{W}{W},{1}{G}{U},{1}{G}{U}{R},{1}{G}{U}{U},{1}{G}{W},{1}{G}{W}{U},{1}{G}{W}{W},{1}{R/G},{1}{R/G}{R/G}{R/G},{1}{R/W},{1}{R/W}{R/W},{1}{R},{1}{R}{G},{1}{R}{G}{G},{1}{R}{G}{W},{1}{R}{R},{1}{R}{R}{R},{1}{R}{R}{W},{1}{R}{W},{1}{R}{W}{B},{1}{R}{W}{W},{1}{U/B},{1}{U/B}{U/B},{1}{U/B}{U/B}{U/B},{1}{U/R},{1}{U/R}{U/R},{1}{U},{1}{U}{B},{1}{U}{B}{B},{1}{U}{B}{R},{1}{U}{R},{1}{U}{R}{W},{1}{U}{U},{1}{U}{U}{B},{1}{U}{U}{R},{1}{U}{U}{U},{1}{W/B},{1}{W/B}{W/B},{1}{W/P},{1}{W/U},{1}{W/U}{W/U},{1}{W/U}{W/U}{W/U},{1}{W},{1}{W}{B},{1}{W}{B}{G},{1}{W}{U},{1}{W}{U}{B},{1}{W}{U}{B}{R}{G},{1}{W}{W},{1}{W}{W}{B}{B},{1}{W}{W}{U},{1}{W}{W}{U}{U},{1}{W}{W}{W},{2/B}{2/B}{2/B},{2/G}{2/G}{2/G},{2/R}{2/R}{2/R},{2/U}{2/U}{2/U},{2/W}{2/U}{2/B}{2/R}{2/G},{2/W}{2/W}{2/W},{2},{2}{B/G},{2}{B/G}{B/G},{2}{B/G}{B/G}{B/G},{2}{B/P}{B/P},{2}{B/R},{2}{B/R}{B/R},{2}{B},{2}{B}{B},{2}{B}{B}{B},{2}{B}{B}{G}{G},{2}{B}{B}{R},{2}{B}{B}{R}{R},{2}{B}{B}{R}{R}{G}{G},{2}{B}{G},{2}{B}{G}{G},{2}{B}{G}{U},{2}{B}{R},{2}{B}{R}{G},{2}{B}{R}{R},{2}{C},{2}{G/U},{2}{G/U}{G/U}{G/U},{2}{G/U}{W},{2}{G/W},{2}{G/W}{G/W},{2}{G/W}{G/W}{G/W},{2}{G},{2}{G}{G},{2}{G}{G}{G},{2}{G}{G}{G}{G},{2}{G}{G}{U},{2}{G}{G}{U}{U},{2}{G}{G}{W},{2}{G}{G}{W}{W},{2}{G}{G}{W}{W}{U}{U},{2}{G}{U},{2}{G}{U}{R},{2}{G}{U}{U},{2}{G}{W},{2}{G}{W}{U},{2}{G}{W}{W},{2}{R/G},{2}{R/G}{R/G},{2}{R/W},{2}{R/W}{G},{2}{R/W}{R/W},{2}{R/W}{R/W}{R/W},{2}{R},{2}{R}{G},{2}{R}{G}{W},{2}{R}{R},{2}{R}{R}{G},{2}{R}{R}{G}{G},{2}{R}{R}{G}{G}{W}{W},{2}{R}{R}{R},{2}{R}{R}{R}{G},{2}{R}{R}{R}{R},{2}{R}{R}{W}{W},{2}{R}{W},{2}{R}{W}{B},{2}{U/B},{2}{U/B}{U/B},{2}{U/P},{2}{U/R},{2}{U/R}{B},{2}{U/R}{U/R},{2}{U},{2}{U}{B},{2}{U}{B}{R},{2}{U}{R},{2}{U}{R}{W},{2}{U}{U},{2}{U}{U}{B},{2}{U}{U}{B}{B},{2}{U}{U}{B}{B}{R}{R},{2}{U}{U}{R}{R},{2}{U}{U}{U},{2}{U}{U}{U}{U},{2}{W/B},{2}{W/B}{U},{2}{W/B}{W/B},{2}{W/B}{W/B}{W/B},{2}{W/P},{2}{W/U},{2}{W/U}{W/U},{2}{W},{2}{W}{B},{2}{W}{B}{G},{2}{W}{U},{2}{W}{U}{B},{2}{W}{U}{U},{2}{W}{U}{U}{B},{2}{W}{W},{2}{W}{W}{B},{2}{W}{W}{B}{B},{2}{W}{W}{U},{2}{W}{W}{U}{U},{2}{W}{W}{U}{U}{B}{B},{2}{W}{W}{W},{3},{3}{B/G},{3}{B/G}{B/G},{3}{B/G}{B/G}{B/G},{3}{B/G}{R},{3}{B/R}{B/R},{3}{B},{3}{B}{B},{3}{B}{B}{B},{3}{B}{B}{B}{B},{3}{B}{B}{G},{3}{B}{B}{R},{3}{B}{B}{R}{R},{3}{B}{G},{3}{B}{G}{U},{3}{B}{R},{3}{B}{R}{G},{3}{C},{3}{G/P},{3}{G/U},{3}{G/U}{G/U},{3}{G/U}{G/U}{G/U},{3}{G/W},{3}{G/W}{G/W},{3}{G},{3}{G}{G},{3}{G}{G}{G},{3}{G}{G}{G}{G},{3}{G}{G}{W},{3}{G}{G}{W}{W},{3}{G}{U},{3}{G}{U}{R},{3}{G}{W},{3}{G}{W}{U},{3}{G}{W}{W},{3}{R/G},{3}{R/P},{3}{R/P}{R/P},{3}{R/W},{3}{R/W}{R/W},{3}{R},{3}{R}{G},{3}{R}{G}{G},{3}{R}{G}{W},{3}{R}{R},{3}{R}{R}{G},{3}{R}{R}{G}{G},{3}{R}{R}{R},{3}{R}{R}{R}{R},{3}{R}{W},{3}{R}{W}{B},{3}{R}{W}{W},{3}{U/B},{3}{U/B}{U/B},{3}{U/B}{U/B}{U/B},{3}{U/P},{3}{U/R},{3}{U/R}{B},{3}{U/R}{U/R}{U/R},{3}{U},{3}{U}{B},{3}{U}{B}{B},{3}{U}{B}{R},{3}{U}{R},{3}{U}{R}{W},{3}{U}{U},{3}{U}{U}{B},{3}{U}{U}{B}{B},{3}{U}{U}{U},{3}{W/B},{3}{W/B}{W/B},{3}{W/B}{W/B}{W/B},{3}{W/P}{W/P},{3}{W/U},{3}{W/U}{W/U},{3}{W/U}{W/U}{W/U},{3}{W},{3}{W}{B},{3}{W}{B}{B},{3}{W}{B}{G},{3}{W}{U},{3}{W}{U}{B},{3}{W}{U}{B}{R}{G},{3}{W}{U}{U},{3}{W}{W},{3}{W}{W}{B}{B},{3}{W}{W}{U},{3}{W}{W}{U}{U},{3}{W}{W}{W},{4},{4}{B/G},{4}{B/G}{B/G}{B/G},{4}{B/P},{4}{B/P}{B/P}{B/P},{4}{B/R},{4}{B},{4}{B}{B},{4}{B}{B}{B},{4}{B}{B}{B}{B},{4}{B}{B}{G}{G},{4}{B}{B}{R}{R},{4}{B}{G},{4}{B}{G}{U},{4}{B}{R},{4}{B}{R}{G},{4}{B}{R}{R},{4}{B}{R}{R}{G},{4}{C},{4}{G/P}{G/P},{4}{G/U}{G/U},{4}{G/U}{G/U}{G/U},{4}{G},{4}{G}{G},{4}{G}{G}{G},{4}{G}{G}{G}{G},{4}{G}{G}{W}{W},{4}{G}{U},{4}{G}{W},{4}{G}{W}{W}{U},{4}{R/G},{4}{R/G}{R

In [29]:
# data['manaCost'].apply(lambda x: 'G' in x)
data['manaCost'] = data['manaCost'].astype(str)

In [30]:
data['manaCost']

0        {2}{G}{G}
1        {1}{U}{U}
2              nan
3           {2}{B}
4           {1}{G}
           ...    
51425    {2}{W}{W}
51426       {3}{U}
51427    {3}{W}{W}
51428       {1}{R}
51429       {4}{G}
Name: manaCost, Length: 51430, dtype: object

In [69]:
#colorless mana cost
data['manaCost'].apply(lambda x: int(x[1:x.find('}')]) if x[1:x.find('}')].isnumeric() else 0)

In [ ]:
data['manaCost'].apply(lambda x: int(x[1:x.find('}')]) if x[1:x.find('}')].isnumeric() else 0)

In [77]:
# data['manaCost'].apply(lambda x: [y for y in [m.start() for m in re.finditer('{+|}+', x)]])
colors = ['R', 'G', 'W', 'B', 'U']
colors_combinations = ['R/W', 'R/G', 'B/R', 'U/R', 'G/W', 'W/B', 'W/U', 'B/G', 'G/U', 'U/B']
life_pay_combinations = ['R/P', 'W/P', 'G/P', 'B/P', 'U/P']

combined = colors + colors_combinations + life_pay_combinations

manacost = np.zeros((51430, 22)) #first 2 for X pay and colorless pay

for i in range(len(data['manaCost'])):
    values = []
    for id in range(0, len(index[i]), 2):
        first = index[i][id]
        second = index[i][id + 1]
        values.append(data['manaCost'].iloc[i][first + 1:second])
    for val in values:
        if val.isnumeric():
            manacost[i, 20] = int(val)
        elif val == 'X':
            manacost[i, 21] = 1
        else:
            try:
                a = combined.index(val)
                manacost[i, a] += 1
            except ValueError:
                pass
        

In [80]:
manacost

array([[0., 2., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 2., ..., 0., 3., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 4., 0.]])

In [38]:
data['name']

0                       Abundance
1             Academy Researchers
2                  Adarkar Wastes
3                         Afflict
4                 Aggressive Urge
                   ...           
51425            Windborne Charge
51426               Windrider Eel
51427               World Queller
51428    Zektar Shrine Expedition
51429           Zendikar Farguide
Name: name, Length: 51430, dtype: object

In [47]:
data['manaCost']

0        {2}{G}{G}
1        {1}{U}{U}
2              nan
3           {2}{B}
4           {1}{G}
           ...    
51425    {2}{W}{W}
51426       {3}{U}
51427    {3}{W}{W}
51428       {1}{R}
51429       {4}{G}
Name: manaCost, Length: 51430, dtype: object

In [81]:
index

0        [0, 2, 3, 5, 6, 8]
1        [0, 2, 3, 5, 6, 8]
2                        []
3              [0, 2, 3, 5]
4              [0, 2, 3, 5]
                ...        
51425    [0, 2, 3, 5, 6, 8]
51426          [0, 2, 3, 5]
51427    [0, 2, 3, 5, 6, 8]
51428          [0, 2, 3, 5]
51429          [0, 2, 3, 5]
Name: manaCost, Length: 51430, dtype: object

In [82]:
abc = data['manaCost'].apply(lambda x: [y for y in [m.start() for m in re.finditer('{+|}+', x)]])
abc

0        [0, 2, 3, 5, 6, 8]
1        [0, 2, 3, 5, 6, 8]
2                        []
3              [0, 2, 3, 5]
4              [0, 2, 3, 5]
                ...        
51425    [0, 2, 3, 5, 6, 8]
51426          [0, 2, 3, 5]
51427    [0, 2, 3, 5, 6, 8]
51428          [0, 2, 3, 5]
51429          [0, 2, 3, 5]
Name: manaCost, Length: 51430, dtype: object

In [83]:
index

0        [0, 2, 3, 5, 6, 8]
1        [0, 2, 3, 5, 6, 8]
2                        []
3              [0, 2, 3, 5]
4              [0, 2, 3, 5]
                ...        
51425    [0, 2, 3, 5, 6, 8]
51426          [0, 2, 3, 5]
51427    [0, 2, 3, 5, 6, 8]
51428          [0, 2, 3, 5]
51429          [0, 2, 3, 5]
Name: manaCost, Length: 51430, dtype: object